In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
import PIL
import matplotlib.pyplot as plt
import os
from pathlib import Path
from detection import load_detector
import shutil
from progressbar import progressbar
import time
from database import Image, Crop, Bird
from tools import fancy_loop, visualize_detection_boxes, crop_box_to_square_crop_box, detection_box_to_crop_box
from detection import detect_birds
from peewee import JOIN


Ignore failed crops:

In [14]:
image_name = "2b5be06424"
Crop.update({Crop.ignore: 1}).where(Crop.source == Image.get(Image.name == image_name)).execute()

1

In [8]:
[detect_fn, category_index] = load_detector()

Loading model...Done! Took 143.95254826545715s


# Run detector to get bounding boxes

In [9]:
query = (Image
         .select()
         .where(
             (Image.is_duplicate.is_null()) | (Image.is_duplicate == 0), 
             Image.ignore == 0,
             Image.bird.is_null(False),
             (Image.detection_score.is_null()) | (Image.detection_score >= 0.6)
         )
         .join(Crop, JOIN.LEFT_OUTER)
         .where(Crop.id.is_null())
        )
query.count()    

157

In [10]:
for image_entry in fancy_loop(query):
    image_pil = image_entry.load()
    image_np = np.array(image_pil)
    detections = detect_birds(image_np, detect_fn, min_score=0)
    
    high_score = max([d.score for d in detections]) if len(detections) > 0 else 0
    summed_score = max([d.score for d in detections]) if len(detections) > 0 else 0
    image_entry.detection_score = high_score
    image_entry.detection_score_sum = summed_score
    image_entry.save()
    
    if high_score < 0.6:
        continue
    
    detection = max(detections, key=lambda x: x.score)                
    box = detection_box_to_crop_box(detection.box, image_size=image_pil.size)    
    crop_entry = Crop(source=image_entry, x1=box.left, y1=box.top, x2=box.right, y2=box.bottom)
    crop_entry.save()
    

100% (157 of 157) |######################| Elapsed Time: 0:01:13 Time:  0:01:13


In [ ]:
Image.peak(50)

In [ ]:
for crop_entry in Crop.select().offset(3300).limit(10):
    source_image_np = np.array(crop_entry.source.load())
    
    plt.title(str(crop_entry.source.id))
    plt.imshow(source_image_np)
    plt.show()
    
    x1, y1, x2, y2 = crop_box_to_square_crop_box((crop_entry.x1, crop_entry.x2, crop_entry.y1, crop_entry.y2), (crop_entry.source.width, crop_entry.source.height))
    
    print(f"{full_width=}, {full_height=}, {size=}, {x1=}, {x2=}, {y1=}, {y2=}")
    
    crop_image_np = source_image_np[y1:y2, x1:x2]
    
    plt.imshow(crop_image_np)
    plt.show()
    

In [19]:
Crop.delete().where(Crop.source == Image.get(Image.id == 521)).execute()

1